## Imports

In [ ]:
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u

from ccdproc import ImageFileCollection
from tqdm import tqdm

In [ ]:
path_to_spirou_sky_data = '/path/to/spirou_sky_data/'
collec = ImageFileCollection(path_to_spirou_sky_data,
                             keywords=["OBJECT","OBJNAME","TELALT","TELAZ","OBJRA","OBJDEC",
                             "DPRTYPE","TRG_TYPE","MJDMID"], glob_include="*.fits", ext=0)

In [ ]:
table = collec.summary

In [ ]:
time = table['MJDMID'] - int(table['MJDMID'].min())

In [ ]:
mask1_1 = (time_centered > 504) & (time_centered < 504.5)
mask1_2 = (time_centered > 504.58) & (time_centered < 505.515)
mask1_3 = (time_centered > 505.58) & (time_centered < 506.578)
mask1_4 = (time_centered > 506.61) & (time_centered < 507)
mask1 = mask1_1 | mask1_2 | mask1_3 | mask1_4
mask2 = (time_centered > 543) & (time_centered < 547)

## Coords

In [ ]:
plt.figure(figsize=[10,10])
plt.scatter(table['TELAZ'][~(mask1|mask2)], table['TELALT'][~(mask1|mask2)], label='All observations', alpha=0.1, s=100)
plt.scatter(table['TELAZ'][mask2], table['TELALT'][mask2], color='C1', marker='^', label='Event 2 observations', alpha=0.1, s=100)
plt.scatter(table['TELAZ'][mask1], table['TELALT'][mask1], color='C2', marker='h', label='Event 1 observations', alpha=0.1, s=100)
plt.title('Sky Observations Altitude-Azimuth Plot', fontsize=20)
plt.xlabel('Azimuth (degrees)', fontsize=20)
plt.ylabel('Altitude (degrees)', fontsize=20)
plt.tick_params(labelsize=20)
plt.legend(fontsize=10)

## Median Sky

In [ ]:
path_wave = '../data/spirou_sky_mean_median_stacks.npz'
wave = np.load(path_wave)['wavelength']
flux_med = np.load(path_wave)['flux_median']
flux_mean = np.load(path_wave)['flux_mean']
med = np.nanmedian(flux_med)
flux_scale = flux_med / med

In [ ]:
mask_j = (wave > 1.17) & (wave < 1.37)
mask_h = (wave > 1.47) & (wave < 1.70)
mask_k = (wave > 1.95) & (wave < 2.35)
mask_not = ~(mask_j|mask_h|mask_k)

In [ ]:
lines_vac, lines_air = np.loadtxt('../data/hydroxyl_lines_rousselot_2000.txt').T / 10000

In [ ]:
plt.figure(figsize=[30,10])
plt.title('SPIRou Median Sky Spectra', fontsize=30)
plt.scatter(wave[mask_not], flux_scale[mask_not], s=1)
plt.scatter(wave[mask_not][1000], flux_scale[mask_not][1000], s=50, label='data', color='C0')
plt.scatter(wave[mask_j], flux_scale[mask_j], s=1)
plt.scatter(wave[mask_j][0], flux_scale[mask_j][0], s=50, label='J Band', color='C1')
plt.scatter(wave[mask_h], flux_scale[mask_h], s=1)
plt.scatter(wave[mask_h][7500], flux_scale[mask_h][7500], s=50, label='H Band', color='C2')
plt.scatter(wave[mask_k], flux_scale[mask_k], s=1)
plt.scatter(wave[mask_k][8000], flux_scale[mask_k][8000], s=50, label='K Band', color='C3')
plt.vlines(lines_vac, 0, 250, color='k', alpha=0.1, label='OH line')
plt.xlabel('Wavelength ($\mu$m)', fontsize=30)
plt.ylabel('Relative Flux', fontsize=30)
plt.tick_params(labelsize=30)
plt.legend(fontsize=30)